In [1]:
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from loader import get_train_loader

In [74]:
dimZ = 256

class Autoencoder8(nn.Module):
    def __init__(self):
        super(Autoencoder8, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=2, padding=0),
#             nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=2, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=2, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(32, 16, kernel_size=2, padding=1),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16, 16, 2, stride=2),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ConvTranspose2d(16, 16, 2, stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 3, kernel_size=3, padding=1),
            nn.Sigmoid(),
        )
        
    def forward(self, x):
        latent_code = self.encoder(x)
        reconstruction = self.decoder(latent_code)
        return reconstruction, latent_code

In [75]:
# Create MSE loss function
criterion = torch.nn.MSELoss()

autoencoder = Autoencoder8().cuda()

# Use Adam optimizer
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)

In [12]:
train_loader, val_loader = get_train_loader("../deepfashion/index.p", batch_size=64, resize_size=16)

In [ ]:
# Train your autoencoder
# Visualize progress in reconstruction and loss decay

train_loss = []
val_loss = []
reconstructed_pictures = []

import time
num_epochs = 100 # total amount of full passes over training data

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    start_time = time.time()
    autoencoder.train(True) # enable dropout / batch_norm training behavior
    i = 0
    for X_batch, y_batch in train_loader:
        i += 1
        # train on batch
        X_batch_0 = torch.FloatTensor(X_batch[:, 0])
        X_batch_0 = Variable(X_batch_0).cuda()
#         X_batch = Variable(X_batch)
#         y_batch = Variable(y_batch)
        output_img, _ = autoencoder(X_batch_0)
        loss = criterion(output_img, X_batch_0)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss.append(loss.cpu().data.numpy()[0])
#         train_loss.append(loss.data.numpy())
        
        X_batch_1 = torch.FloatTensor(X_batch[:, 1])
        X_batch_1 = Variable(X_batch_1).cuda()
#         X_batch = Variable(X_batch)
#         y_batch = Variable(y_batch)
        output_img, _ = autoencoder(X_batch_1)
        loss = criterion(output_img, X_batch_1)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss.append(loss.cpu().data.numpy()[0])
#         train_loss.append(loss.data.numpy())
#         print(i, ":", loss.data.cpu().numpy()[0], end=", ")

    # And a full pass over the validation data:
    autoencoder.train(False) # disable dropout / use averages for batch_norm
    for X_batch, y_batch in val_loader:
        X_batch_0 = Variable(X_batch[:, 0]).cuda()
        output_img, _ = autoencoder(X_batch_0)
        val_loss.append(criterion(output_img, X_batch_0).cpu().data.numpy()[0])
        X_batch_1 = Variable(X_batch[:, 1]).cuda()
        output_img, _ = autoencoder(X_batch_1)
        val_loss.append(criterion(output_img, X_batch_1).cpu().data.numpy()[0])
#         val_loss.append(criterion(output_img, X_batch).data.numpy())
#     if epoch % 16 == 0:
#         X_batch = Variable(torch.FloatTensor(np.array([X_val[247]]))).cuda()
#         output_img, _ = autoencoder(X_batch)
#         # reconstructed_pictures.append(output_img.cpu().data.numpy()[0])
#         reconstructed_pictures.append(output_img.data.numpy())
    # Then we print the results for this epoch:
    print 
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss (in-iteration): \t{:.6f}".format(
        np.mean(train_loss[-2 * len(train_loader):])))
    print("  validation loss: \t\t\t{:.6f}".format(
        np.mean(val_loss[-2 * len(val_loader):])))

Epoch 1 of 100 took 61.297s
  training loss (in-iteration): 	0.001223
  validation loss: 			0.001159
Epoch 2 of 100 took 61.529s
  training loss (in-iteration): 	0.001213
  validation loss: 			0.001160


In [77]:
def plot_gallery(images, h, w, n_row=3, n_col=6):
    """Helper function to plot a gallery of portraits"""
    scale_const = 1.2
    plt.figure(figsize=(3 / scale_const * n_col, 3.4 / scale_const * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].numpy().transpose(1,2,0), cmap=plt.cm.gray, vmin=-1, vmax=1, interpolation='nearest')
        plt.xticks(())
        plt.yticks(())

In [ ]:
autoencoder.train(False)

In [ ]:
plot_gallery(X_batch[:3, 1], 16, 16, n_row=1, n_col=3)

In [ ]:
plot_gallery(X_batch[:3, 0], 16, 16, n_row=1, n_col=3)

In [ ]:
plot_gallery(autoencoder(Variable(X_batch[:3, 1]).cuda())[0].cpu().data, 16, 16, n_row=1, n_col=3)

In [ ]:
torch.save(autoencoder, "autoencoder.people.8.pt")

# Train stickmans

In [ ]:
# Create MSE loss function
criterion = torch.nn.MSELoss()

autoencoder = Autoencoder8().cuda()

# Use Adam optimizer
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)

In [94]:
# Train your autoencoder
# Visualize progress in reconstruction and loss decay

train_loss = []
val_loss = []
reconstructed_pictures = []

import time
num_epochs = 1 # total amount of full passes over training data

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    start_time = time.time()
    autoencoder.train(True) # enable dropout / batch_norm training behavior
    i = 0
    for X_batch, y_batch in train_loader:
        i += 1
        # train on batch
        X_batch_0 = torch.FloatTensor(y_batch[:, 0])
        X_batch_0 = Variable(X_batch_0).cuda()
#         X_batch = Variable(X_batch)
#         y_batch = Variable(y_batch)
        output_img, _ = autoencoder(X_batch_0)
        loss = criterion(output_img, X_batch_0)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss.append(loss.cpu().data.numpy()[0])
#         train_loss.append(loss.data.numpy())
        
        X_batch_1 = torch.FloatTensor(y_batch[:, 1])
        X_batch_1 = Variable(X_batch_1).cuda()
#         X_batch = Variable(X_batch)
#         y_batch = Variable(y_batch)
        output_img, _ = autoencoder(X_batch_1)
        loss = criterion(output_img, X_batch_1)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss.append(loss.cpu().data.numpy()[0])
#         train_loss.append(loss.data.numpy())
#         print(i, ":", loss.data.cpu().numpy()[0], end=", ")

    # And a full pass over the validation data:
    autoencoder.train(False) # disable dropout / use averages for batch_norm
    for X_batch, y_batch in val_loader:
        X_batch_0 = Variable(y_batch[:, 0]).cuda()
        output_img, _ = autoencoder(X_batch_0)
        val_loss.append(criterion(output_img, X_batch_0).cpu().data.numpy()[0])
        X_batch_1 = Variable(y_batch[:, 1]).cuda()
        output_img, _ = autoencoder(X_batch_1)
        val_loss.append(criterion(output_img, X_batch_1).cpu().data.numpy()[0])
#         val_loss.append(criterion(output_img, X_batch).data.numpy())
#     if epoch % 16 == 0:
#         X_batch = Variable(torch.FloatTensor(np.array([X_val[247]]))).cuda()
#         output_img, _ = autoencoder(X_batch)
#         # reconstructed_pictures.append(output_img.cpu().data.numpy()[0])
#         reconstructed_pictures.append(output_img.data.numpy())
    # Then we print the results for this epoch:
    print 
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss (in-iteration): \t{:.6f}".format(
        np.mean(train_loss[-2 * len(train_loader):])))
    print("  validation loss: \t\t\t{:.6f}".format(
        np.mean(val_loss[-2 * len(val_loader):])))

Epoch 1 of 1 took 60.342s
  training loss (in-iteration): 	0.000253
  validation loss: 			0.000271


In [ ]:
autoencoder.train(False)

In [ ]:
torch.save(autoencoder, "autoencoder.stickmen.8.pt")

In [ ]:
plot_gallery(y_batch[:3, 1], image_h, image_w, n_row=1, n_col=3)

In [ ]:
plot_gallery(autoencoder(Variable(y_batch[:3, 1]).cuda())[0].cpu().data, image_h, image_w, n_row=1, n_col=3)